<a href="https://colab.research.google.com/github/Coden-inja/colab-pipeline-test/blob/main/testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# --- BLOCK 1: SETUP ENVIRONMENT ---
print("⚙️ Installing Dependencies...")
!apt-get update -qq && apt-get install -y ffmpeg
!pip install ffmpeg-python
import sys
import os
import subprocess
import glob

⚙️ Installing Dependencies...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.


In [ ]:
# --- BLOCK 2: CREATE MOCK DATA ---
print("\n🎥 Generating Synthetic Test Video (10s, 30fps)...")
# Generates a 10s video with a clock and test pattern
subprocess.run([
    'ffmpeg', '-y', '-f', 'lavfi',
    '-i', 'testsrc=duration=10:size=1920x1080:rate=30',
    '-c:v', 'libx264', '-pix_fmt', 'yuv420p', 'mock_input.mp4'
], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

if os.path.exists('mock_input.mp4'):
    print("✅ mock_input.mp4 created.")
else:
    print("❌ Failed to create mock video.")

In [5]:
# --- STEP 2: IMPORT & PATCH (THE FIX) ---
# Ensure we can find the uploaded file
sys.path.append(os.getcwd())

try:
    from video_processor import VideoProcessor
except ImportError:
    print("\n❌ ERROR: 'video_processor.py' not found.")
    print("👉 Please look at the file menu on the left and upload 'video_processor.py' first.")
    sys.exit(1)

print("\n🚀 Initializing Processor...")
processor = VideoProcessor()

# *** CRITICAL FIX FOR COLAB ***
# We manually overwrite the hardware config to force CPU.
# This prevents the "h264_nvenc" crash you just saw.
processor.hardware_config = {
    'c:v': 'libx264',
    'preset': 'veryfast',
    'crf': 23,
    'pix_fmt': 'yuv420p'
}
print("🔧 Forced CPU Mode (Bypassing Colab GPU mismatch)...")

# --- STEP 3: RUN THE PIPELINE ---
output_dir = "./output_test_final"
try:
    processor.process('mock_input.mp4', output_dir)
    print("\n✅ SUCCESS! Transcoding finished without crashing.")
except Exception as e:
    print(f"\n❌ FAILED: {e}")
    sys.exit(1)

# --- STEP 4: VERIFY RESULTS ---
print("\n🔍 Verifying HLS Chunks...")
m3u8_path = os.path.join(output_dir, 'index.m3u8')
poster_path = os.path.join(output_dir, 'poster.jpg')
chunks = glob.glob(os.path.join(output_dir, '*.ts'))

print(f"   • Playlist: {'✅ Found' if os.path.exists(m3u8_path) else '❌ Missing'}")
print(f"   • Poster:   {'✅ Found' if os.path.exists(poster_path) else '❌ Missing'}")
print(f"   • Chunks:   {len(chunks)} generated (Target: ~5)")

if len(chunks) > 0:
    import ffmpeg
    try:
        probe = ffmpeg.probe(chunks[0])
        duration = float(probe['format']['duration'])
        print(f"   • Chunk 0 Duration: {duration}s")
        if 1.9 <= duration <= 2.1:
            print("   • Quality Check: ✅ PERFECT (Chunks are exactly 2.0s)")
        else:
            print("   • Quality Check: ⚠️ DRIFT (Chunks are not aligned)")
    except:
        print("   • Could not probe chunk duration.")


🚀 Initializing Processor...
🔧 Forced CPU Mode (Bypassing Colab GPU mismatch)...
{"status": "success", "hls_url": "/videos/output_test_final/index.m3u8", "poster_url": "/videos/output_test_final/poster.jpg"}

✅ SUCCESS! Transcoding finished without crashing.

🔍 Verifying HLS Chunks...
   • Playlist: ✅ Found
   • Poster:   ✅ Found
   • Chunks:   5 generated (Target: ~5)
   • Chunk 0 Duration: 2.0s
   • Quality Check: ✅ PERFECT (Chunks are exactly 2.0s)
